In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dask

%matplotlib inline

In [3]:
reader = pd.read_csv('data/news_cleaned.csv', usecols=['id', 'domain', 'type', 'content'], iterator=True)

In [4]:
df1 = reader.get_chunk(50000)

In [5]:
df1.head()

,id,domain,type,content
0,2,express.co.uk,rumor,"Life is an illusion, at least on a quantum lev..."
1,6,barenakedislam.com,hate,"Unfortunately, he hasn’t yet attacked her for ..."
2,7,barenakedislam.com,hate,The Los Angeles Police Department has been den...
3,8,barenakedislam.com,hate,The White House has decided to quietly withdra...
4,9,barenakedislam.com,hate,“The time has come to cut off the tongues of t...


In [6]:
df1['type'].unique()

array(['rumor', 'hate', 'unreliable', 'conspiracy', 'clickbait', 'satire',
       'fake', 'reliable', 'bias', 'political', 'junksci', nan, 'unknown'], dtype=object)

In [7]:
len(df1.dropna())

46708

In [8]:
len(df1.drop('id', axis=1).dropna())

46708

In [9]:
len(df1.drop('domain', axis=1).dropna())

46708

In [10]:
df1 = df1.dropna()

In [11]:
df1['type'].unique()

array(['rumor', 'hate', 'unreliable', 'conspiracy', 'clickbait', 'satire',
       'fake', 'reliable', 'bias', 'political', 'junksci', 'unknown'], dtype=object)

In [12]:
len(df1[(df1['type'] != 'unreliable') & (df1['type'] !='unknown')])

45038

In [13]:
df1 = df1[(df1['type'] != 'unreliable') & (df1['type'] !='unknown')]

In [14]:
df1.groupby('type').size()

type
bias           1224
clickbait       267
conspiracy     5297
fake          25420
hate            272
junksci        1361
political     10489
reliable        273
rumor            96
satire          339
dtype: int64

In [16]:
df1[df1['type'] == 'political']['domain'].unique()

array(['baptistnews.com', 'nationalreview.com', 'dailycaller.com',
       'rinf.com', 'washingtonexaminer.com', 'breitbart.com',
       'judicialwatch.org', 'guardianlv.com', 'city-journal.org',
       'ecowatch.com', 'weeklystandard.com', 'pjmedia.com',
       'mintpressnews.com', 'freedomworks.org', 'alternet.org',
       'conservativereview.com', 'yellowhammernews.com',
       'filmsforaction.org', 'rawstory.com', 'theintercept.com',
       'commondreams.org', 'dailykos.com', 'thinkprogress.org',
       'thedailybeast.com', 'counterpunch.org', 'observer.com',
       'americannewsx.com', 'americanprogress.org', 'ronpaulinstitute.org',
       'theblaze.com', 'jacobinmag.com', 'foreignpolicyjournal.com',
       'heritage.org', 'countercurrents.org', 'newcoldwar.org',
       'nakedcapitalism.com', 'commentarymagazine.com', 'redstate.com'], dtype=object)

In [19]:
media_bias = pd.read_csv('data/media_bias.csv')

In [21]:
media_bias[media_bias['Vertical Rank'] >= 40]

,News Source,Vertical Rank,Horizontal Rank
0,ABC,57,0
1,AFP,62,0
2,Al Jazeera US/Canada News,54,-1
4,AP,62,0
5,Axios,52,-2
6,BBC,54,-3
8,Bloomberg,58,4
11,BuzzFeed News,51,-15
12,CBS,57,4
13,Christian Science Monitor,54,6


In [22]:
import scrapy
import re
from scrapy.crawler import CrawlerProcess

In [30]:
some_data = pd.read_json('quotes.json')

In [31]:
some_data.loc[0, 'article']

'BUENOS AIRES, Argentina (AP) — Broken dates. Coy suggestions. Missed encounters. Private opportunities. Amid weighty issues of state, another fascination at the Group of 20 summit has been President Donald Trump’s will-he, won’t-he dance with two fellow leaders who are something of international outcasts these days. Would Trump, who has an affinity for strongmen and a distaste for business as usual, stay away from Russian President Vladimir Putin and Saudi Crown Prince Mohammed bin Salman? On Day One of the summit in Argentina — fittingly, the land of the tango — the diplomatic quick-stepping was everywhere Friday. Trump has not been shy in his praise of the crown prince and Putin in the past. But Prince Mohammed has been under global pressure lately over the murder of a Saudi journalist, and Putin has drawn fresh criticism for his country’s mounting aggression against Ukraine. So Trump canceled his plans to meet with Putin and left bin Salman off his public agenda. But even then, Tru

In [29]:
len(some_data)

55